## Load Data from Mongo collection and json files

In this file we will combine all the datasets together.
Variable ```mixed_sample_events``` will contain all the extracted data.


The domain names are modified for anonymity.

In [ ]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from collections import defaultdict
import statistics
from urllib.parse import urlparse
import re

def connect_to_mongodb(connection_string="mongodb://mongo_server:27017/"):
    """Connect to MongoDB database"""
    try:
        client = MongoClient(connection_string)
        # Test the connection
        client.admin.command('ping')
        print("✓ Connected to MongoDB successfully")
        return client
    except Exception as e:
        print(f"✗ Failed to connect to MongoDB: {e}")
        return None

def extract_domain_from_url(url):
    """Extract domain from URL, handling various URL formats"""
    try:
        # Handle file:// URLs
        if url.startswith('file:///'):
            # Extract the drive/path pattern for file URLs
            match = re.match(r'file:///([A-Za-z]:/[^/]*)', url)
            if match:
                return f"file:///{match.group(1)}"
            return "file:///local"
        
        # Handle regular URLs
        parsed = urlparse(url)
        if parsed.netloc:
            # Remove www. prefix for cleaner grouping
            domain = parsed.netloc.lower()
            if domain.startswith('www.'):
                domain = domain[4:]
            return f"{parsed.scheme}://{domain}"
        
        # Fallback for malformed URLs
        return url.split('/')[0] if '/' in url else url
    
    except Exception:
        return url

def load_mongodb_to_dataframe(db_name="test", collection_name="userIds"):
    """Load MongoDB collection into pandas DataFrame"""
    
    # Connect to MongoDB
    client = connect_to_mongodb()
    if not client:
        return None
    
    try:
        # Access database and collection
        db = client[db_name]
        collection = db[collection_name]
        
        print(f"Loading data from collection: {db_name}.{collection_name}")
        print("-" * 60)
        
        # Get all documents
        documents = list(collection.find({}))
        
        if not documents:
            print("No documents found in the collection")
            return None
        
        print(f"Loaded {len(documents)} documents from MongoDB")
        
        # Convert to DataFrame
        df = pd.DataFrame(documents)
        
        return df
        
    except Exception as e:
        print(f"Error loading data: {e}")
        return None
    
    finally:
        client.close()

# Load the data
print("MongoDB User Data Analysis - Pandas Version")
print("=" * 50)

df = load_mongodb_to_dataframe("test", "userIds")
if df is not None:
    print(f"\nDataFrame shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\nFirst few rows:")
    display(df.head())

# Expand URLs from metadata and create a detailed analysis DataFrame
if df is not None:
    print("Expanding URLs from metadata...")
    
    # Create a list to store expanded data
    expanded_data = []
    
    for idx, row in df.iterrows():
        user_id = row.get('_id', 'unknown')
        
        # Extract URLs from metadata
        if 'metadata' in row and isinstance(row['metadata'], dict) and 'urls' in row['metadata']:
            urls = row['metadata']['urls']
            if isinstance(urls, list):
                url_list = urls
            elif isinstance(urls, str):
                url_list = [urls]
            else:
                url_list = []
            
            # Add each URL as a separate row
            for url in url_list:
                domain = extract_domain_from_url(url)
                
                expanded_data.append({
                    'user_id': user_id,
                    'full_url': url,
                    'domain': domain,
                    'event_count': row['metadata'].get('eventCount', 0) if 'metadata' in row else 0,
                    'first_timestamp': row['metadata'].get('firstTimestamp', '') if 'metadata' in row else '',
                    'last_timestamp': row['metadata'].get('lastTimestamp', '') if 'metadata' in row else '',
                    'user_agent': row['metadata'].get('fingerprint', {}).get('userAgent', '') if 'metadata' in row and 'fingerprint' in row['metadata'] else '',
                    'source_ip': row['metadata'].get('fingerprint', {}).get('source', '') if 'metadata' in row and 'fingerprint' in row['metadata'] else ''
                })
    
    # Create expanded DataFrame
    url_df = pd.DataFrame(expanded_data)
    
    print(f"\nExpanded DataFrame created!")
    print(f"Shape: {url_df.shape}")
    print(f"Total URLs: {len(url_df)}")
    print(f"Unique users: {url_df['user_id'].nunique()}")
    print(f"Unique domains: {url_df['domain'].nunique()}")
    
    display(url_df.head(10))



# Fixed Domain Analysis with proper formatting
if 'url_df' in locals() and url_df is not None:
    print("🔍 DOMAIN ANALYSIS (FIXED)")
    print("=" * 60)
    
    # Group by domain and calculate statistics
    domain_stats = url_df.groupby('domain').agg({
        'user_id': 'nunique',  # Count unique users
        'full_url': 'count',   # Count total URLs
        'event_count': 'sum'   # Sum event counts
    }).rename(columns={
        'user_id': 'unique_users',
        'full_url': 'total_urls',
        'event_count': 'total_events'
    })
    
    # Calculate percentages
    total_urls = len(url_df)
    total_users = url_df['user_id'].nunique()
    
    domain_stats['url_percentage'] = (domain_stats['total_urls'] / total_urls * 100).round(1)
    domain_stats['user_percentage'] = (domain_stats['unique_users'] / total_users * 100).round(1)
    
    # Sort by total URLs (most frequent first)
    domain_stats = domain_stats.sort_values('total_urls', ascending=False)
    
    print(f"\n📊 Summary Statistics:")
    print(f"Total URLs processed: {total_urls:,}")
    print(f"Total unique users: {total_users:,}")
    print(f"Unique domains found: {len(domain_stats):,}")
    
    print(f"\n🔝 Top Domains by URL Frequency:")
    print("-" * 100)
    print(f"{'Rank':<4} {'Domain':<45} {'URLs':<8} {'Users':<8} {'Events':<10} {'URL %':<8} {'User %'}")
    print("-" * 100)
    
    # Fixed formatting - convert to int before using :d format
    for i, (domain, stats) in enumerate(domain_stats.head(20).iterrows(), 1):
        total_urls_int = int(stats['total_urls'])
        unique_users_int = int(stats['unique_users']) 
        total_events_int = int(stats['total_events'])
        print(f"{i:3d}. {domain:<45} {total_urls_int:6d}   {unique_users_int:6d}   {total_events_int:8d}   {stats['url_percentage']:5.1f}%   {stats['user_percentage']:5.1f}%")
    
    # Store the results for further analysis
    domain_analysis_df = domain_stats.reset_index()
    
    print(f"\n💾 Domain analysis saved to 'domain_analysis_df' variable")
    display(domain_analysis_df.head(10))


# Event Fetching Pipeline - Get detailed events from events collection
def fetch_user_events(user_ids, db_name="test", collection_name="events", limit_per_user=None, min_events=1000):
    """
    Fetch detailed events for specific users from the events collection
    
    Args:
        user_ids: List of user IDs to fetch events for
        db_name: Database name
        collection_name: Events collection name
        limit_per_user: Maximum events per user (optional)
    
    Returns:
        DataFrame with all events for the specified users
    """
    
    # Connect to MongoDB
    client = connect_to_mongodb()
    if not client:
        return None
    
    try:
        db = client[db_name]
        collection = db[collection_name]
        
        print(f"Fetching events for {len(user_ids)} users from {db_name}.{collection_name}")
        
        # Build query for multiple users
        query = {"userId": {"$in": user_ids}}
        
        # Fetch events
        events = []
        
        if limit_per_user:
            # Fetch limited events for each user individually
            for user_id in user_ids:
                user_query = {"userId": user_id}
                user_events = list(collection.find(user_query)
                                #  .sort("timestamp", 1)  # Sort by timestamp ascending
                                 .limit(limit_per_user))
                if len(user_events) > min_events:
                    events.extend(user_events)
                else:
                    print(f"No events found for user {user_id}")
                
            print(f"Fetched up to {limit_per_user} events per user")
        else:
            # Fetch all events for all users
            events = list(collection.find(query))
        
        print(f"Found {len(events)} events")
        
        if not events:
            return pd.DataFrame()
        
        # Convert to DataFrame
        events_df = pd.DataFrame(events)
        
        # Convert timestamp to numeric for analysis
        if 'timestamp' in events_df.columns:
            events_df['timestamp_num'] = pd.to_numeric(events_df['timestamp'], errors='coerce')
            events_df['timestamp_dt'] = pd.to_datetime(events_df['timestamp_num'], unit='ms', errors='coerce')
        
        # Sort by user and timestamp
        events_df = events_df.sort_values(['userId', 'timestamp_num'])
        
        return events_df
        
    except Exception as e:
        print(f"Error fetching events: {e}")
        return None
    
    finally:
        client.close()



# User Filtering Pipeline - Extract userIds based on domain/URL criteria
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns



def break_into_sessions(user_events, session_gap_minutes=5, max_sessions=500):
    """
    Break user events into separate sessions based on time gaps
    
    Args:
        user_events: DataFrame with events for a single user
        session_gap_minutes: Minutes of inactivity to consider a new session
    
    Returns:
        List of session DataFrames
    """
    if len(user_events) == 0:
        return []
    
    # Sort by timestamp
    user_events = user_events.sort_values('timestamp_num').copy()
    
    # Calculate time differences in minutes
    user_events['time_diff_minutes'] = user_events['timestamp_num'].diff() / (1000 * 60)
    
    # Identify session breaks (gaps larger than session_gap_minutes)
    session_breaks = user_events['time_diff_minutes'] > session_gap_minutes
    
    # Create session IDs
    user_events['session_id'] = session_breaks.cumsum()
    
    # Split into separate sessions
    sessions = []
    for session_id in user_events['session_id'].unique():
        session_events = user_events[user_events['session_id'] == session_id].copy()
        if len(session_events) >= 2:  # Only include sessions with at least 2 events
            sessions.append(session_events)
            if len(sessions) >= max_sessions:
                print(f"Found {len(sessions)} sessions")
                break
    
    return sessions


def analyze_event_patterns(events_df, session_gap_minutes=5):
    """
    Analyze event patterns for distinguishing features with session segmentation
    
    Args:
        events_df: DataFrame with user events
        session_gap_minutes: Minutes of inactivity to consider a new session
    
    Returns:
        Dictionary with pattern analysis results
    """
    
    if events_df is None or len(events_df) == 0:
        return {}
    
    results = {}
    
    # 1. Event type distribution
    event_types = events_df['eventName'].value_counts()
    results['event_types'] = event_types.to_dict()
    
    # 2. Session-based timing analysis
    user_timing = []
    total_sessions = 0
    
    for user_id in events_df['userId'].unique():
        user_events = events_df[events_df['userId'] == user_id].copy()
        
        if len(user_events) < 2:
            continue
        
        # Break user events into sessions
        sessions = break_into_sessions(user_events, session_gap_minutes)
        
        if not sessions:
            continue
        
        total_sessions += len(sessions)
        
        # Analyze each session separately
        user_session_data = []
        
        for i, session_events in enumerate(sessions):
            # Calculate intervals between events within this session
            session_events = session_events.sort_values('timestamp_num')
            session_events['time_diff'] = session_events['timestamp_num'].diff()
            
            # Remove first row (NaN) and convert to milliseconds
            intervals = session_events['time_diff'].dropna()
            
            if len(intervals) > 0:
                session_duration = (session_events['timestamp_num'].max() - 
                                  session_events['timestamp_num'].min()) / 1000  # Convert to seconds
                
                # Calculate consistency metrics
                intervals_seconds = intervals / 1000  # Convert to seconds
                avg_interval = intervals_seconds.mean()
                
                # Coefficient of Variation (CV) - normalized measure of variability
                cv_interval = (intervals_seconds.std() / avg_interval) if avg_interval > 0 else 0
                
                # Calculate trend in intervals (are they getting faster/slower?)
                if len(intervals_seconds) >= 3:
                    # Linear regression slope to detect trend
                    x = np.arange(len(intervals_seconds))
                    slope = np.polyfit(x, intervals_seconds, 1)[0]
                    interval_trend = slope  # Positive = getting slower, Negative = getting faster
                    
                    # R-squared to measure how linear the trend is
                    correlation = np.corrcoef(x, intervals_seconds)[0, 1]
                    trend_strength = correlation ** 2 if not np.isnan(correlation) else 0
                else:
                    interval_trend = 0
                    trend_strength = 0
                
                # Consistency categories
                if cv_interval < 0.3:
                    consistency_level = "Very Consistent"
                elif cv_interval < 0.6:
                    consistency_level = "Moderately Consistent"
                elif cv_interval < 1.0:
                    consistency_level = "Variable"
                else:
                    consistency_level = "Highly Variable"
                
                # Pattern detection within session
                intervals_array = intervals_seconds.values
                
                # Detect acceleration/deceleration patterns
                if len(intervals_array) >= 5:
                    first_half_avg = intervals_array[:len(intervals_array)//2].mean()
                    second_half_avg = intervals_array[len(intervals_array)//2:].mean()
                    pace_change = (second_half_avg - first_half_avg) / first_half_avg * 100
                else:
                    pace_change = 0
                
                session_data = {
                    'user_id': user_id,
                    'session_number': i + 1,
                    'total_events': len(session_events),
                    'session_duration': session_duration,
                    'avg_interval': avg_interval,
                    'median_interval': intervals_seconds.median(),
                    'min_interval': intervals_seconds.min(),
                    'max_interval': intervals_seconds.max(),
                    'std_interval': intervals_seconds.std(),
                    
                    # NEW: Consistency metrics
                    'cv_interval': cv_interval,  # Coefficient of variation
                    'consistency_level': consistency_level,
                    'interval_trend': interval_trend,  # Slope of interval change
                    'trend_strength': trend_strength,  # How linear the trend is (0-1)
                    'pace_change_percent': pace_change,  # % change from first to second half
                    
                    'event_types': session_events['eventName'].value_counts().to_dict(),
                    'start_time': session_events['timestamp_dt'].min(),
                    'end_time': session_events['timestamp_dt'].max()
                }
                
                user_session_data.append(session_data)
        
        # Add all sessions for this user
        user_timing.extend(user_session_data)
    
    results['user_timing'] = pd.DataFrame(user_timing)
    results['total_sessions'] = total_sessions
    results['session_gap_minutes'] = session_gap_minutes
    
    print(f"    📊 Broke {len(events_df['userId'].unique())} users into {total_sessions} sessions")
    print(f"    ⏱️ Session gap threshold: {session_gap_minutes} minutes")
    
    # 3. Common event sequences (within sessions)
    sequences = []
    for user_id in events_df['userId'].unique():
        user_events = events_df[events_df['userId'] == user_id].copy()
        
        # Break into sessions first
        sessions = break_into_sessions(user_events, session_gap_minutes)
        
        # Extract sequences from each session separately
        for session_events in sessions:
            session_events = session_events.sort_values('timestamp_num')
            event_sequence = session_events['eventName'].tolist()
            
            # Create 3-event sequences within this session
            for i in range(len(event_sequence) - 2):
                seq = tuple(event_sequence[i:i+3])
                sequences.append(seq)
    
    if sequences:
        from collections import Counter
        sequence_counts = Counter(sequences)
        results['common_sequences'] = dict(sequence_counts.most_common(20))
    
    # 4. URL patterns
    if 'url' in events_df.columns:
        url_patterns = events_df.groupby('userId')['url'].nunique()
        results['urls_per_user'] = {
            'mean': url_patterns.mean(),
            'median': url_patterns.median(),
            'min': url_patterns.min(),
            'max': url_patterns.max()
        }
    
    return results


def filter_users_by_criteria(url_df, domain_filters=None, url_filters=None, min_events=None, max_events=None, max_users=None):
    """
    Filter users based on domain or URL criteria and optional event count thresholds
    
    Args:
        url_df: DataFrame with user URL data
        domain_filters: List of domains to include 
        url_filters: List of full URLs to include
        min_events: Minimum number of events per user (optional)
        max_events: Maximum number of events per user (optional)
        max_users: Maximum number of users to return (optional)
    
    Returns:
        DataFrame with filtered users and their metadata
    """
    url_df = url_df[url_df['user_id'] != '0']
    # Filter by domains or URLs
    if domain_filters and url_filters:
        # Filter by both domains AND URLs (union)
        domain_mask = url_df['domain'].isin(domain_filters)
        url_mask = url_df['full_url'].isin(url_filters)
        filtered_df = url_df[domain_mask | url_mask].copy()
        print(f"    🔍 Filtering by {len(domain_filters)} domains AND {len(url_filters)} specific URLs")
    elif domain_filters:
        # Filter by domains only
        filtered_df = url_df[url_df['domain'].isin(domain_filters)].copy()
        print(f"    🔍 Filtering by {len(domain_filters)} domains")
    elif url_filters:
        # Filter by specific URLs only
        filtered_df = url_df[url_df['full_url'].isin(url_filters)].copy()
        print(f"    🔍 Filtering by {len(url_filters)} specific URLs")
    else:
        # No filters - use all data
        filtered_df = url_df.copy()
        print(f"    🔍 No filters applied - using all data")
    
    # Group by user and aggregate
    user_summary = filtered_df.groupby('user_id').agg({
        'domain': lambda x: list(x.unique()),  # List of domains visited
        'full_url': 'count',  # Number of URLs visited
        'event_count': 'first',  # Event count from metadata
        'first_timestamp': 'first',
        'last_timestamp': 'first',
        'user_agent': 'first',
        'source_ip': 'first'
    }).rename(columns={'full_url': 'url_count'})
    
    # Apply event count filters if specified
    if min_events is not None:
        user_summary = user_summary[user_summary['event_count'] >= min_events]
    
    if max_events is not None:
        user_summary = user_summary[user_summary['event_count'] <= max_events]
    
    # Calculate session duration
    user_summary['session_duration'] = 0
    for idx, row in user_summary.iterrows():
        if row['first_timestamp'] and row['last_timestamp']:
            try:
                first_ts = int(row['first_timestamp'])
                last_ts = int(row['last_timestamp'])
                
                user_summary.loc[idx, 'session_duration'] = (last_ts - first_ts) / 1000
            except (ValueError, TypeError):
                pass
    
    # Apply user limit if specified
    result_df = user_summary.reset_index()
    if max_users is not None and len(result_df) > max_users:
        # Sort by event count (descending) to get most active users first
        result_df = result_df.sort_values('event_count', ascending=False).head(max_users)
        print(f"    ⚠️ Limited to top {max_users} users (sorted by event count)")
    
    return result_df

# Example usage - filter users by specific domains
if 'url_df' in locals():
    print("🎯 USER FILTERING PIPELINE")
    print("=" * 50)
    
    # Define advanced domain groups with URL-specific filtering
    advanced_domain_groups = {
        'tax1_users': {
            'url_filters': [
                # 'https://PLACEHOLDER_TAX1.com'
            ],
            # 'domain_filters': [''],
            'description': 'All Tax1 domain users'
        },
        'tax2_users': {
            'url_filters': [
                ''
            ],
            # 'domain_filters': ['https://PLACEHOLDER_TAX2.com'],
            'description': 'All Tax2 domain users'
        },
        'profilic_survey_users': {
            'url_filters': [
                #'https://PLACEHOLDER_SURVEY.com'
            ],
            'description': 'Profilic Survey users'
        },
        'browser_use_agents': {
            'url_filters': [
                'https://AGENT_SURVVEY_PLACEHOLDER.com/?agent=browser-use'
            ],
            'description': 'Browser Use Agents'
        },
        'skyvern_agents': {
            'url_filters': [
                'https://AGENT_SURVVEY_PLACEHOLDER.com/?agent=skyvern'
            ],
            'description': 'Skyvern Agents'
        },
    }

    max_users = {
        # 'tax1_users': 20,
        # 'tax2_users': 20,
        # 'profilic_survey_users': 20,
        # 'browser_use_agents': 20,
        # 'skyvern_agents': 20,
    }
    
    # Filter users for each advanced group
    user_groups = {}
    
    for group_name, criteria in advanced_domain_groups.items():
        print(f"\n🎯 Processing {group_name.upper()}: {criteria['description']}")
        
        # Extract filtering parameters
        domain_filters = criteria.get('domain_filters', None)
        url_filters = criteria.get('url_filters', None)
        
        # Skip if no valid URLs found (e.g., for tax_2023_users if no matches)
        if url_filters is not None and len(url_filters) == 0:
            print(f"    ⚠️ No matching URLs found for {group_name}, skipping...")
            continue
        
        # Limit to 30 users per group, minimum 10 events, get most active users
        filtered_users = filter_users_by_criteria(
            url_df, 
            domain_filters=domain_filters,
            url_filters=url_filters,
            min_events=10, 
            max_users=max_users.get(group_name, None)
        )
        
        user_groups[group_name] = filtered_users
        
        print(f"📊 Results for {group_name.upper()}:")
        print(f"  • Total users: {len(filtered_users):,}")
        if len(filtered_users) > 0:
            print(f"  • Avg events per user: {filtered_users['event_count'].mean():.1f}")
            print(f"  • Avg session duration: {filtered_users['session_duration'].mean():.1f}s")
            print(f"  • Sample user IDs: {list(filtered_users['user_id'].head(3))}")
            
            # Show some sample URLs for URL-filtered groups
            if url_filters and len(url_filters) > 0:
                print(f"  • Sample URLs matched: {url_filters[:3]}")
        else:
            print(f"  ⚠️ No users found matching criteria")
    
    print(f"\n💾 User groups saved to 'user_groups' dictionary")

# File-based User Group Loading
import os
import json
import glob

def load_file_based_user_groups(base_directory, file_extension=".json"):
    """
    Load user groups from directory structure where each subdirectory represents a group
    
    Args:
        base_directory: Base path containing group subdirectories (e.g., "forensics_data/")
        file_extension: File extension to look for (default: ".json")
    
    Returns:
        Dictionary with group names and user IDs from files
    """
    
    print(f"📁 LOADING FILE-BASED USER GROUPS")
    print(f"=" * 50)
    print(f"Base directory: {base_directory}")
    
    file_user_groups = {}
    
    if not os.path.exists(base_directory):
        print(f"⚠️ Directory {base_directory} does not exist")
        return file_user_groups
    
    # Get all subdirectories
    subdirs = [d for d in os.listdir(base_directory) 
               if os.path.isdir(os.path.join(base_directory, d))]
    
    print(f"Found {len(subdirs)} group directories: {subdirs}")
    
    for group_dir in subdirs:
        group_path = os.path.join(base_directory, group_dir)
        
        # Find all JSON files in this directory
        json_pattern = os.path.join(group_path, f"*{file_extension}")
        json_files = glob.glob(json_pattern)
        
        if not json_files:
            print(f"⚠️ No {file_extension} files found in {group_dir}")
            continue
        
        # Extract user IDs from filenames
        user_ids = []
        valid_files = []
        
        for json_file in json_files:
            filename = os.path.basename(json_file)
            # Remove extension to get user ID
            user_id = filename.replace(file_extension, "")
            
            # Validate that file is readable
            try:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                    if isinstance(data, dict) and 'trace' in data:
                        user_ids.append(user_id)
                        valid_files.append(json_file)
            except (json.JSONDecodeError, IOError) as e:
                print(f"⚠️ Skipping invalid file {filename}: {e}")
                continue
        
        if user_ids:
            file_user_groups[group_dir] = {
                'user_ids': user_ids,
                'file_paths': valid_files,
                'source': 'file',
                'description': f'File-based group from {group_dir} directory',
                'count': len(user_ids)
            }
            
            print(f"\n📊 {group_dir.upper()}:")
            print(f"  • Total users: {len(user_ids)}")
            print(f"  • Sample user IDs: {user_ids[:3]}")
            print(f"  • Source: File-based ({group_path})")
        else:
            print(f"⚠️ No valid users found in {group_dir}")
    
    return file_user_groups

def load_events_from_file(file_path):
    """
    Load events from a JSON file with forensics data structure
    
    Expected format:
    {
        "user_id": "0a5f288e-5220-4c83-aa3b-976a9bbca2d1",
        "device": "desktop", 
        "trace": [
            {"event_name": "mousemove", "timestamp": "1667342141173", "position": {"x": 1298, "y": 257}},
            {"event_name": "mousedown", "timestamp": "1667342141555", "position": {"x": 1300, "y": 257}},
            ...
        ]
    }
    
    Args:
        file_path: Path to the JSON file
    
    Returns:
        DataFrame with events in standardized format
    """
    
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Handle forensics data structure
        if isinstance(data, dict) and 'trace' in data:
            # Extract metadata
            user_id = data.get('user_id', 'unknown')
            device = data.get('device', 'unknown')
            
            # Get events from trace
            events = data['trace']
            
            if not events:
                return pd.DataFrame()
            
            # Convert to DataFrame
            events_df = pd.DataFrame(events)
            
            # Standardize column names to match MongoDB format
            column_mapping = {
                'event_name': 'eventName',
                'timestamp': 'timestamp'
            }
            
            events_df = events_df.rename(columns=column_mapping)
            
            # Add user metadata
            events_df['userId'] = user_id
            events_df['device'] = device
            
            # Handle position data - convert to pos format like MongoDB
            if 'position' in events_df.columns:
                events_df['pos'] = events_df['position'].apply(
                    lambda x: json.dumps(x) if isinstance(x, dict) else '{"x":0,"y":0}'
                )
            
            # Ensure timestamp is numeric
            if 'timestamp' in events_df.columns:
                events_df['timestamp_num'] = pd.to_numeric(events_df['timestamp'], errors='coerce')
                events_df['timestamp_dt'] = pd.to_datetime(events_df['timestamp_num'], unit='ms', errors='coerce')
            
            # Add missing columns with default values to match MongoDB structure
            default_columns = {
                'cursor': 0,
                'element': '/',
                'attrs': '',
                'extra': '{"trusted":true}',
                'url': f'file:///{device}_forensics_data',
                'dimensions': '{"screenw":1920,"screenh":1080,"winw":1920,"winh":945,"docw":1903,"doch":2542}',
                'fingerprint': f'{{"source":"forensics_data","userAgent":"ForensicsData/{device}"}}'
            }
            
            for col, default_val in default_columns.items():
                if col not in events_df.columns:
                    events_df[col] = default_val
            
            return events_df
            
        # Fallback for other formats
        elif isinstance(data, list):
            # Direct list of events
            events = data
        elif isinstance(data, dict):
            if 'events' in data:
                # Events nested under 'events' key
                events = data['events']
            else:
                # Assume the dict itself represents events
                events = [data]
        else:
            return pd.DataFrame()
        
        # Convert to DataFrame (fallback path)
        events_df = pd.DataFrame(events)
        
        # Ensure required columns exist
        if 'timestamp' in events_df.columns:
            events_df['timestamp_num'] = pd.to_numeric(events_df['timestamp'], errors='coerce')
            events_df['timestamp_dt'] = pd.to_datetime(events_df['timestamp_num'], unit='ms', errors='coerce')
        
        return events_df
        
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return pd.DataFrame()

def show_user_group_statistics(mongo_user_groups, file_user_groups):
    """
    Display detailed statistics about available users in each group
    """
    print(f"📊 USER GROUP STATISTICS")
    print(f"=" * 50)
    
    total_mongo_users = 0
    total_file_users = 0
    
    if mongo_user_groups:
        print(f"\n🗄️ MongoDB Groups:")
        for group_name, users_df in mongo_user_groups.items():
            user_count = len(users_df)
            total_mongo_users += user_count
            avg_events = users_df['event_count'].mean() if 'event_count' in users_df.columns else 0
            print(f"  • {group_name:<25} {user_count:>6} users (avg {avg_events:.0f} events)")
    
    if file_user_groups:
        print(f"\n📁 File-based Groups:")
        for group_name, group_info in file_user_groups.items():
            user_count = group_info['count']
            total_file_users += user_count
            print(f"  • {group_name:<25} {user_count:>6} users (from files)")
    
    print(f"\n📈 Summary:")
    print(f"  • Total MongoDB users: {total_mongo_users:,}")
    print(f"  • Total file-based users: {total_file_users:,}")
    print(f"  • Grand total: {total_mongo_users + total_file_users:,} users")

def fetch_events_by_url(url, db_name="test", collection_name="events", limit_per_user=None, min_events=1000):
    """
    Fetch detailed events for specific users from the events collection
    
    Args:
        url: List of user IDs to fetch events for
        db_name: Database name
        collection_name: Events collection name
        limit_per_user: Maximum events per user (optional)
    
    Returns:
        DataFrame with all events for the specified users
    """
    # Connect to MongoDB
    client = connect_to_mongodb()
    if not client:
        return None
    
    try:
        db = client[db_name]
        collection = db[collection_name]
        
        print(f"Fetching events for {len(url)} url from {db_name}.{collection_name}")

        query = {"url": url}
        
        # Build query for multiple users
        events = list(collection.find(query))
        
        print(f"Found {len(events)} events")
        
        if not events:
            return pd.DataFrame()
        
        # Convert to DataFrame
        events_df = pd.DataFrame(events)
        
        # Convert timestamp to numeric for analysis
        if 'timestamp' in events_df.columns:
            events_df['timestamp_num'] = pd.to_numeric(events_df['timestamp'], errors='coerce')
            events_df['timestamp_dt'] = pd.to_datetime(events_df['timestamp_num'], unit='ms', errors='coerce')
        
        # Sort by user and timestamp
        events_df = events_df.sort_values(['userId', 'timestamp_num'])
        
        return events_df
        
    except Exception as e:
        print(f"Error fetching events: {e}")
        return None
    
    finally:
        client.close()


def create_mixed_sample_events(mongo_user_groups, file_user_groups, max_users_per_group=100):
    """
    Create sample events from both MongoDB and file-based sources
    
    Args:
        mongo_user_groups: User groups from MongoDB (existing format)
        file_user_groups: User groups from files
        max_users_per_group: Maximum users to process per group
    
    Returns:
        Combined sample_events dictionary
    """
    
    print(f"🔄 CREATING MIXED SAMPLE EVENTS")
    print(f"=" * 50)
    print(f"Max users per group: {max_users_per_group}")
    
    # Show statistics first
    show_user_group_statistics(mongo_user_groups, file_user_groups)
    
    mixed_sample_events = {}
    
    # Process MongoDB groups (existing logic)
    for group_name, users_df in mongo_user_groups.items():
        if len(users_df) > 0:
            sample_size = min(max_users_per_group, len(users_df))
            sample_users = users_df.head(sample_size)['user_id'].tolist()
            
            print(f"\n🎯 Fetching MongoDB events for {group_name} ({sample_size} users)...")

            # Fetch events from MongoDB
            if group_name == 'open_operator_agents':
                events_df = fetch_events_by_url(url='https://agents.cheapernyhomes.com/?agent=openoperator')
            # Fetch events from MongoDB
            else:
                events_df = fetch_user_events(sample_users, limit_per_user=10000)

            
            
            if events_df is not None and len(events_df) > 0:
                patterns = analyze_event_patterns(events_df)
                
                mixed_sample_events[group_name] = {
                    'events_df': events_df,
                    'patterns': patterns,
                    'user_count': sample_size,
                    'total_events': len(events_df),
                    'source': 'mongodb'
                }
                
                print(f"  ✓ Found {len(events_df):,} events from MongoDB")
    
    # Process file-based groups
    for group_name, group_info in file_user_groups.items():
        user_ids = group_info['user_ids']
        file_paths = group_info['file_paths']
        
        sample_size = min(max_users_per_group, len(user_ids))
        sample_users = user_ids[:sample_size]
        sample_files = file_paths[:sample_size]
        
        print(f"\n🎯 Loading file events for {group_name} ({sample_size} users)...")
        
        # Load events from files
        all_events = []
        successful_loads = 0
        
        for i, (user_id, file_path) in enumerate(zip(sample_users, sample_files)):
            # Show progress for larger datasets
            if i > 0 and i % 10 == 0:
                print(f"    📈 Processed {i}/{len(sample_users)} files...")

            events_df = load_events_from_file(file_path)
            
            if not events_df.empty:
                # Add user ID if not present
                if 'userId' not in events_df.columns:
                    events_df['userId'] = user_id
                
                all_events.append(events_df)
                successful_loads += 1
            else:
                print(f"    ⚠️ No events loaded from {os.path.basename(file_path)}")
        
        if all_events:
            # Combine all events
            combined_events_df = pd.concat(all_events, ignore_index=True)
            
            # Analyze patterns
            patterns = analyze_event_patterns(combined_events_df)
            
            mixed_sample_events[group_name] = {
                'events_df': combined_events_df,
                'patterns': patterns,
                'user_count': successful_loads,
                'total_events': len(combined_events_df),
                'source': 'file'
            }
            
            print(f"  ✓ Found {len(combined_events_df):,} events from {successful_loads} files")
        else:
            print(f"  ⚠️ No valid events found for {group_name}")
    
    print(f"\n📊 Mixed Sample Events Summary:")
    for group_name, data in mixed_sample_events.items():
        source = data.get('source', 'unknown')
        print(f"  • {group_name}: {data['user_count']} users, {data['total_events']:,} events ({source})")
    
    return mixed_sample_events

# # Example usage - load file-based groups
# print("\n" + "="*70)
# print("📁 FILE-BASED USER GROUP INTEGRATION")
# print("="*70)

# Load file-based user groups
base_dir = "forensics_data"  # Change this to your actual path
file_user_groups = load_file_based_user_groups(base_dir)

# Combine with existing MongoDB groups if available
if 'user_groups' in locals() and file_user_groups:
    print(f"\n🔄 Combining MongoDB and file-based groups...")
    
    # Create mixed sample events with more users
    mixed_sample_events = create_mixed_sample_events(user_groups, file_user_groups, max_users_per_group=100)
    
    print(f"\n💾 Mixed sample events saved to 'mixed_sample_events' variable")
    print(f"📋 Available groups: {list(mixed_sample_events.keys())}")
    
elif file_user_groups:
    print(f"\n📁 File-based groups loaded:")
    for group_name, info in file_user_groups.items():
        print(f"  • {group_name}: {info['count']} users")
    
    # Create sample events from files only with more users
    mixed_sample_events = create_mixed_sample_events({}, file_user_groups, max_users_per_group=100)
    
    print(f"\n💾 File-based sample events saved to 'mixed_sample_events' variable")
else:
    print(f"⚠️ No file-based groups found. Check the base directory path.")
    print(f"Expected structure:")
    print(f"  forensics_data/")
    print(f"  ├── gremlins/")
    print(f"  │   ├── user1.json")
    print(f"  │   └── user2.json")
    print(f"  └── other_group/")
    print(f"      ├── user3.json")
    print(f"      └── user4.json")


